<a href="https://colab.research.google.com/github/Kartik8625/credit-default-prediction-random-forest/blob/main/credit_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>